# Deep Learning Toolkit for Splunk - Dask Distributed KMeans

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client
import dask_ml.cluster

# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

/opt/conda/lib/python3.7/site-packages/dask/array/random.py:27: FutureWarning: dask.array.random.doc_wraps is deprecated and will be removed in a future version
  FutureWarning,


In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
import dask_ml
print("dask_ml version: " + dask_ml.__version__)

dask_ml version: 1.2.0


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| inputlookup iris.csv <br/>
| fit MLTKContainer algo=dask_kmeans k=3 petal_length petal_width sepal_length sepal_width into app:iris_dask_kmeans

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [3]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [4]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("iris_dask_kmeans")
print(df)
print(param)

     sepal_length  sepal_width  petal_length  petal_width
0             5.1          3.5           1.4          0.2
1             4.9          3.0           1.4          0.2
2             4.7          3.2           1.3          0.2
3             4.6          3.1           1.5          0.2
4             5.0          3.6           1.4          0.2
..            ...          ...           ...          ...
145           6.7          3.0           5.2          2.3
146           6.3          2.5           5.0          1.9
147           6.5          3.0           5.2          2.0
148           6.2          3.4           5.4          2.3
149           5.9          3.0           5.1          1.8

[150 rows x 4 columns]
{'options': {'params': {'mode': 'stage', 'algo': 'dask_kmeans', 'k': '3'}, 'args': ['petal_length', 'petal_width', 'sepal_length', 'sepal_width'], 'feature_variables': ['petal_length', 'petal_width', 'sepal_length', 'sepal_width'], 'model_name': 'iris_dask_kmeans', 'algo_name': '

## Stage 2 - create and initialize a model

In [5]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    
    #client = Client("tcp://127.0.0.1:")
    client = Client(processes=False)
    
    model['dask_client'] = client
    return model

In [17]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{'dask_client': <Client: 'inproc://172.17.0.2/79/10' processes=1 threads=4, memory=2.09 GB>}


## Stage 3 - fit the model

In [18]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    ddf = dd.from_pandas(df, npartitions=4)  
    # features dataframe
    features = ddf[param['feature_variables']]
    #features.persist()
    k = int(param['options']['params']['k'])
    model['dask_kmeans'] = dask_ml.cluster.KMeans(n_clusters=k, init_max_iter=2, oversampling_factor=10)
    model['dask_kmeans'].fit(features)
    return model

In [19]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
%time print(fit(model,df,param))

{'dask_client': <Client: 'inproc://172.17.0.2/79/10' processes=1 threads=4, memory=2.09 GB>, 'dask_kmeans': KMeans(algorithm='full', copy_x=True, init='k-means||', init_max_iter=2,
       max_iter=300, n_clusters=3, n_jobs=1, oversampling_factor=10,
       precompute_distances='auto', random_state=None, tol=0.0001)}
CPU times: user 828 ms, sys: 118 ms, total: 946 ms
Wall time: 1.03 s


## Stage 4 - apply the model

In [20]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    #ddf = dd.from_pandas(df[param['feature_variables']], npartitions=4)
    prediction = model['dask_kmeans'].labels_
    #y_hat = prediction.to_dask_dataframe()
    result = pd.DataFrame(prediction.compute())
    model['dask_client'].close()
    return result

In [21]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
%time print(apply(model,df,param))

     0
0    1
1    1
2    1
3    1
4    1
..  ..
145  0
146  2
147  0
148  0
149  2

[150 rows x 1 columns]
CPU times: user 113 ms, sys: 30.5 ms, total: 144 ms
Wall time: 2.12 s


## Stage 5 - save the model

In [11]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    # TODO
    # model['dask_kmeans'].save_model(MODEL_DIRECTORY + name + '.json')
    return model

In [12]:
save(model,'dask_kmeans')

{'dask_client': <Client: not connected>,
 'dask_kmeans': KMeans(algorithm='full', copy_x=True, init='k-means||', init_max_iter=2,
        max_iter=300, n_clusters=3, n_jobs=1, oversampling_factor=10,
        precompute_distances='auto', random_state=None, tol=0.0001)}

## Stage 6 - load the model

In [13]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    # TODO
    model = {}
    return model

In [14]:
loaded_model = load('dask_kmeans')
loaded_model

{}

## Stage 7 - provide a summary of the model

In [15]:
# return a model summary
def summary(model=None):
    returns = {"model": model}
    return returns

In [16]:
summary(model)

{'model': {'dask_client': <Client: not connected>,
  'dask_kmeans': KMeans(algorithm='full', copy_x=True, init='k-means||', init_max_iter=2,
         max_iter=300, n_clusters=3, n_jobs=1, oversampling_factor=10,
         precompute_distances='auto', random_state=None, tol=0.0001)}}

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code